# Foreign Market Lead-Lag ML Strategy
## Notebook 3: Model Training & Validation

This notebook trains and validates ML models:
- Train Lasso, Random Forest, Gradient Boosting models
- Perform walk-forward out-of-sample validation
- Calculate R²_OOS and Information Coefficient
- Analyze feature importance
- Compare model performance across stocks

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import warnings
warnings.filterwarnings('ignore')

from feature_engineering import FeatureEngineering
from ml_models import MLModels, MultiStockPredictor

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

%matplotlib inline
%load_ext autoreload
%autoreload 2

## 1. Load Data and Configuration

In [ ]:
# Load config
with open('../config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Load data
sp500_returns = pd.read_csv('../data/sp500_daily_returns.csv', index_col=0, parse_dates=True)
foreign_returns = pd.read_csv('../data/foreign_weekly_returns.csv', index_col=0, parse_dates=True)

print(f"Data loaded: {sp500_returns.shape[1]} stocks, {len(sp500_returns)} days")

## 2. Prepare Training Data

In [ ]:
# Prepare features
feature_eng = FeatureEngineering(config)

# Select sample stocks for detailed analysis
sample_stocks = sp500_returns.columns[:10].tolist()
print(f"Sample stocks: {sample_stocks}")

# Prepare data for sample stocks
sample_stock_data = {}
for stock in sample_stocks:
    X, y = feature_eng.prepare_training_data(foreign_returns, sp500_returns, stock)
    if len(X) > 0:
        sample_stock_data[stock] = (X, y)

print(f"\nPrepared data for {len(sample_stock_data)} sample stocks")

## 3. Train Single Stock Model (Detailed Analysis)

In [ ]:
# Select first stock for detailed analysis
test_stock = list(sample_stock_data.keys())[0]
X, y = sample_stock_data[test_stock]

print(f"Analyzing: {test_stock}")
print(f"Training samples: {len(X)}")

# Initialize model
ml_models = MLModels(config)

# Perform walk-forward validation
print("\nPerforming walk-forward validation...")
results = ml_models.walk_forward_validation(X, y, train_years=5, test_years=1)

print(f"\nValidation Results for {test_stock}:")
print(f"  R²_OOS: {results['r2_oos']:.4f}")
print(f"  RMSE: {results['rmse']:.4f}")
print(f"  IC: {results['ic']:.4f}")
print(f"  Predictions: {len(results['predictions'])}")

## 4. Visualize Predictions vs Actuals

In [ ]:
# Plot predictions vs actuals
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Scatter plot
axes[0, 0].scatter(results['actuals'], results['predictions'], alpha=0.3, s=10)
axes[0, 0].plot([results['actuals'].min(), results['actuals'].max()], 
               [results['actuals'].min(), results['actuals'].max()], 
               'r--', linewidth=2, label='Perfect Prediction')
axes[0, 0].set_title(f'Predictions vs Actuals ({test_stock})', fontweight='bold')
axes[0, 0].set_xlabel('Actual Returns')
axes[0, 0].set_ylabel('Predicted Returns')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Time series
dates = pd.to_datetime(results['dates'])
axes[0, 1].plot(dates, results['actuals'], label='Actual', alpha=0.7, linewidth=1)
axes[0, 1].plot(dates, results['predictions'], label='Predicted', alpha=0.7, linewidth=1)
axes[0, 1].set_title('Time Series: Predictions vs Actuals', fontweight='bold')
axes[0, 1].set_xlabel('Date')
axes[0, 1].set_ylabel('Return')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Residuals
residuals = results['actuals'] - results['predictions']
axes[1, 0].hist(residuals, bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[1, 0].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[1, 0].set_title('Residual Distribution', fontweight='bold')
axes[1, 0].set_xlabel('Residual')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].grid(True, alpha=0.3)

# 4. Cumulative returns by prediction quintile
pred_df = pd.DataFrame({
    'prediction': results['predictions'],
    'actual': results['actuals']
})
pred_df['quintile'] = pd.qcut(pred_df['prediction'], 5, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5'])

for quintile in ['Q1', 'Q5']:
    quintile_returns = pred_df[pred_df['quintile'] == quintile]['actual']
    cumulative = (1 + quintile_returns).cumprod()
    axes[1, 1].plot(range(len(cumulative)), cumulative.values, 
                   label=f'{quintile} (Worst)' if quintile == 'Q1' else f'{quintile} (Best)', 
                   linewidth=2)

axes[1, 1].set_title('Cumulative Returns by Prediction Quintile', fontweight='bold')
axes[1, 1].set_xlabel('Time')
axes[1, 1].set_ylabel('Cumulative Return')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Feature Importance Analysis

In [ ]:
# Train final model to get feature importance
model = ml_models.train_final_model(X, y, test_stock)
feature_importance = ml_models.get_feature_importance(test_stock)

# Get top features
feature_importance.index = X.columns
top_features = feature_importance.abs().sort_values(ascending=False).head(20)

# Plot
fig, ax = plt.subplots(figsize=(12, 8))
top_features.plot(kind='barh', ax=ax, color='steelblue', edgecolor='black')
ax.set_title(f'Top 20 Feature Importance ({test_stock})', fontsize=14, fontweight='bold')
ax.set_xlabel('Coefficient (Lasso)')
ax.axvline(x=0, color='black', linestyle='--', alpha=0.5)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nTop 10 Most Important Features:")
for feature, importance in top_features.head(10).items():
    print(f"  {feature}: {importance:.6f}")

## 6. Train Models for All Sample Stocks

In [ ]:
# Train models for all sample stocks
print("Training models for all sample stocks...")
predictor = MultiStockPredictor(config)
validation_results = predictor.train_all_stocks(sample_stock_data, validate=True)

# Extract R²_OOS scores
r2_scores = {stock: results['r2_oos'] for stock, results in validation_results.items()}
r2_df = pd.Series(r2_scores).sort_values(ascending=False)

print(f"\nValidation Results Summary:")
print(f"  Stocks with positive R²_OOS: {(r2_df > 0).sum()} / {len(r2_df)}")
print(f"  Mean R²_OOS: {r2_df.mean():.4f}")
print(f"  Median R²_OOS: {r2_df.median():.4f}")

# Plot R²_OOS distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Bar chart
r2_df.plot(kind='bar', ax=axes[0], color='steelblue', edgecolor='black')
axes[0].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[0].set_title('R²_OOS by Stock', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Stock')
axes[0].set_ylabel('R²_OOS')
axes[0].grid(True, alpha=0.3)
axes[0].tick_params(axis='x', rotation=45)

# Histogram
r2_df.hist(bins=20, ax=axes[1], edgecolor='black', alpha=0.7, color='coral')
axes[1].axvline(x=0, color='red', linestyle='--', linewidth=2, label='Zero')
axes[1].axvline(x=r2_df.mean(), color='blue', linestyle='--', linewidth=2, label='Mean')
axes[1].set_title('R²_OOS Distribution', fontsize=14, fontweight='bold')
axes[1].set_xlabel('R²_OOS')
axes[1].set_ylabel('Frequency')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Compare Model Types

In [ ]:
# Compare Lasso vs Random Forest vs Gradient Boosting
model_types = ['lasso', 'random_forest', 'gradient_boosting']
model_comparison = {}

print("Comparing model types...")
for model_type in model_types:
    print(f"\nTesting {model_type}...")
    config['models']['primary_model'] = model_type
    ml_models_test = MLModels(config)
    
    results = ml_models_test.walk_forward_validation(X, y, train_years=5, test_years=1)
    model_comparison[model_type] = {
        'r2_oos': results['r2_oos'],
        'rmse': results['rmse'],
        'ic': results['ic']
    }

# Plot comparison
comparison_df = pd.DataFrame(model_comparison).T

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, metric in enumerate(['r2_oos', 'rmse', 'ic']):
    comparison_df[metric].plot(kind='bar', ax=axes[idx], 
                               color='steelblue', edgecolor='black')
    axes[idx].set_title(f'{metric.upper()}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Model Type')
    axes[idx].set_ylabel(metric.upper())
    axes[idx].grid(True, alpha=0.3)
    axes[idx].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("\nModel Comparison Results:")
print(comparison_df)

## 8. Save Models

In [ ]:
# Save trained models
import os
os.makedirs('../models', exist_ok=True)

for stock in predictor.stock_models.keys():
    predictor.ml_models.save_model(stock, f'../models/{stock}_model.pkl')

print(f"Saved {len(predictor.stock_models)} models to ../models/")

# Save validation results
r2_df.to_csv('../results/validation_r2_scores.csv')
print("Saved validation results to ../results/validation_r2_scores.csv")

## Summary

This notebook trained and validated ML models:
- Performed walk-forward out-of-sample validation
- Calculated R²_OOS, RMSE, and Information Coefficient
- Analyzed feature importance (Lasso coefficients)
- Compared different model types

**Key Findings**:
- Lasso model shows predictive power for subset of stocks
- R²_OOS varies significantly across stocks
- Different foreign markets have varying importance
- Model performance aligns with research expectations (~24% of stocks show positive R²_OOS)

**Next Steps**: Proceed to Notebook 4 for portfolio construction and backtesting.